In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import scipy.stats as vale
import seaborn as sns
from sklearn import linear_model


Considera la información anexa en este correo. Revisa los tres conjuntos de datos y determina si es necesario realizar una limpieza determinando los siguientes puntos:
¿Cuáles columnas tienen valores nulos? ¿Por qué tienen valores nulos? ¿Hay alguna manera de eliminarlos?
¿La información de los departamentos es válida? ¿Existen departamentos cuyas ventas son irregulares? ¿Sería válido eliminar estos departamentos del estudio cuando no se estudian estos de manera específica?
¿Cómo es el comportamiento de venta de las tiendas? ¿Existe alguna tienda atípica o con información pobre que se deba eliminar?

In [ ]:
features_data= pd.read_csv("Features data set.csv",parse_dates=["Date"])
sales_data= pd.read_csv("sales data-set.csv",parse_dates=["Date"])
stores_data=pd.read_csv("stores data-set.csv")
print(sales_data)
print(features_data)
print(stores_data)

In [ ]:
stores_data.isna().sum()


In [ ]:
revision_nulos= features_data.iloc[:,[1,0,10]].drop_duplicates()
#print(revision_nulos)

#for i in range(revision_nulos.shape[0]):
    #print(revision_nulos.iloc[[i]])
    
revision_nulos.pivot(index="Store", columns="Date", values= "Unemployment").isna().sum()


In [ ]:
grafica= sales_data.groupby(["Dept","Date"]).sum().reset_index()
depto_nulos=grafica.pivot(index="Date",columns="Dept",values="Weekly_Sales").isna().sum()
depto_nulos[depto_nulos>0]
sns.lineplot(x="Date",y="Weekly_Sales", data= grafica[grafica["Dept"]==1])


In [ ]:
valor_tiendas= sales_data.iloc[:,[0,2,3]]
valor_tiendas= valor_tiendas.groupby(["Store","Date"]).sum().reset_index()
#print(valor_tiendas)
sns.lineplot(x="Date",y="Weekly_Sales", data= valor_tiendas,hue="Store")
valor_tiendas.pivot(index="Date", columns="Store", values= "Weekly_Sales").max().sort_values()


## Fase 1: Entendimiento de los datos

Distribución de Pareto por Tipo de Tienda

In [ ]:

ventas_tienda1= sales_data.merge(stores_data,on="Store")
print(ventas_tienda1)
#print(sales_data)
#print(stores_data)

In [ ]:
ventas_tienda=ventas_tienda1.iloc[:,[3,5]]
ventas_tienda=ventas_tienda.groupby(["Type"]).sum().reset_index()
total_ventas=ventas_tienda["Weekly_Sales"].sum()
ventas_tienda["%"]=ventas_tienda["Weekly_Sales"]/total_ventas
#print(ventas_tienda)

ventas_tiendastipo= ventas_tienda1.iloc[:,[0,3,5]]
ventas_tiendastipo=ventas_tiendastipo.groupby(["Store","Type"]).sum()
ventas_tipo=ventas_tiendastipo.groupby(["Type"]).mean()
print(ventas_tipo)
ventas_tipo.plot.bar()


Distribución de Pareto por Tienda

In [ ]:
pareto_tienda=sales_data.iloc[:,[0,3]]
pareto_tienda=pareto_tienda.groupby(["Store"]).sum().sort_values(by="Weekly_Sales", ascending=False)
pareto_tienda["%"]= pareto_tienda["Weekly_Sales"]/total_ventas    
pareto_tienda["acumulado"]= pareto_tienda["%"].cumsum()
total= pareto_tienda["%"].sum()                             
print(pareto_tienda)
pareto_tienda.plot.bar()
total

### 

Distribución de Pareto por Departamento

In [ ]:
pareto_departamento=sales_data.iloc[:,[1,3]]
pareto_departamento["Weekly_Sales"][pareto_departamento["Weekly_Sales"]<0]=0
pareto_departamento=pareto_departamento.groupby(["Dept"]).sum().sort_values(by="Weekly_Sales",ascending=False)
pareto_departamento["%"]=pareto_departamento/total_ventas
pareto_departamento["acumulado"]= pareto_departamento["%"].cumsum()
print(pareto_departamento)
pareto_departamento.plot.bar()


Explora la Distribución de Pareto de la venta por Departamento para cada Tipo de Tienda (es decir, genera un Pareto por cada tipo de tienda). ¿Existe alguna diferencia en el assortment que ofrece cada tipo de tienda, o un cliente puede encontrar todos los productos en todas las tiendas?

In [ ]:
ventas_tienda1

pareto_a=ventas_tienda1[ventas_tienda1["Type"]=="A"]
pareto_a= pareto_a.iloc[:,[1,3]]
totala=pareto_a["Weekly_Sales"].sum()
pareto_a= pareto_a.groupby(["Dept"]).sum().sort_values(by="Weekly_Sales",ascending=False)
pareto_a["%"]=pareto_a["Weekly_Sales"]/totala
pareto_a["acum"]=pareto_a["%"].cumsum()
pareto_a["%"].plot.barh()
print(pareto_a)
print(totala)

In [ ]:
pareto_B=ventas_tienda1[ventas_tienda1["Type"]=="B"]
pareto_B= pareto_B.iloc[:,[1,3]]
totalB=pareto_B["Weekly_Sales"].sum()
pareto_B= pareto_B.groupby(["Dept"]).sum().sort_values(by="Weekly_Sales",ascending=False)
pareto_B["%"]=pareto_B["Weekly_Sales"]/totalB
pareto_B["acum"]=pareto_B["%"].cumsum()
pareto_B["%"].plot.barh()
print(pareto_B)
print(totalB)


In [ ]:
pareto_C=ventas_tienda1[ventas_tienda1["Type"]=="C"]
pareto_C= pareto_C.iloc[:,[1,3]]
totalC=pareto_C["Weekly_Sales"].sum()
pareto_C= pareto_C.groupby(["Dept"]).sum().sort_values(by="Weekly_Sales",ascending=False)
pareto_C["%"]=pareto_C["Weekly_Sales"]/totalC
pareto_C["acum"]=pareto_C["%"].cumsum()
pareto_C["%"].plot.barh()
print(pareto_C)
print()


In [ ]:
print(pareto_a)
print(pareto_B)
print(pareto_C)

## Genera un histograma de las temperaturas de las ciudades en las que se encuentran las tiendas. Estudia, para cada tienda, la temperatura promedio, la temperatura máxima, y la temperatura mínima a la que se han encontrado con gráficas de barras. ¿Existen grupos de tiendas que tengan un comportamiento de clima diferente?

In [ ]:
datos_clima=features_data.iloc[:,[0,1,2]]
datos_clima["año"]= pd.DatetimeIndex(datos_clima["Date"]).year
datos_clima["Centigrados"]= (datos_clima["Temperature"]-32)*(5/9)
datos_clima["Centigrados"].plot.hist()
print(datos_clima)

In [ ]:
clima_media=datos_clima.groupby(["Store"]).mean().sort_values(by="Centigrados",ascending=False)
clima_min=datos_clima.groupby(["Store"]).min().sort_values(by="Centigrados",ascending=False)
#clima_media["Centigrados"].plot.bar()
clima_min["Centigrados"].plot.bar()
print(clima_media)


In [ ]:
sns.lineplot(x="Date",y="Centigrados",hue="Store",data=datos_clima[(datos_clima["Store"]==11)&(datos_clima["año"]==2013)])

##  Genera un histograma de los precios de la gasolina. ¿Entre qué valores parece oscilar?
¿Cómo ha variado su valor a través del tiempo? Realiza un diagrama de dispersión del tiempo contra los precios de gasolina. ¿Existió un cambio importante en el valor promedio?

In [ ]:
precio_gas=features_data.iloc[:,[0,1,3]]
precio_gas["año"]=pd.DatetimeIndex(precio_gas["Date"]).year
#precio_gas["Fuel_Price"].plot.hist()
sns.scatterplot(x="Date",y="Fuel_Price",hue="Store",data=precio_gas)
precio_prom=precio_gas[precio_gas["año"]>=2012]
precio_prom=precio_prom.groupby(["Store"]).mean()
print(precio_prom)

## Genera un histograma del Customer Purchase Index. ¿Qué existe de extraño en este histograma? Genera una gráfica de línea del CPI por tienda. ¿Se puede concluir algo con respecto a la forma del histograma?

In [ ]:
datos_cpi=features_data.iloc[:,[0,1,9]]
datos_cpi["año"]=pd.DatetimeIndex(datos_cpi["Date"]).year
datos_cpi["CPI"].plot.hist()
plt.show()
#sns.lineplot(x="año",y="CPI", data= datos_cpi[datos_cpi["Store"]==7],hue="Store")
sns.lineplot(x="año",y="CPI", data= datos_cpi,hue="Store")
print(datos_cpi)


In [ ]:
cpi_grafica= datos_cpi[datos_cpi["Store"]== 1].dropna()
#cpi_grafica.plot.hist()
print(cpi_grafica)

## Genera nuevas columnas que representen, con un True y False, si el Holiday representa algún evento importante, de acuerdo a la siguiente figura:

In [ ]:
fechas_importancia["superbowl"]=superbowl
fechas_importancia["laborday"]=laborday
fechas_importancia["thanksgiving"]=thanksgiving
fechas_importancia["christmas"]=christmas
fechas_importancia

In [ ]:
features_holidays=features_data.merge(fechas_importancia, on= "Date")
print(features_holidays)

In [ ]:
markdown=features_holidays.iloc[:,[4]].dropna()
markdown.plot.hist()
plt.show()

print(features_holidays["superbowl"])
print(superbowl)
#print(markdown["MarkDown5"].describe())

## Fase2

Considera únicamente la venta anual de las tiendas en el 2011. ¿Existe una diferencia significativa en el promedio de venta anual entre los Tipos de Tienda? Utiliza una prueba de hipótesis con un nivel de significancia del 0.05.

In [ ]:
#features_data= pd.read_csv("Features data set.csv",parse_dates=["Date"])
#sales_data= pd.read_csv("sales data-set.csv",parse_dates=["Date"])
#stores_data=pd.read_csv("stores data-set.csv")
print(sales_data)

In [ ]:
venta_2011= sales_data.merge(stores_data, on="Store")
venta_2011["año"]=pd.DatetimeIndex(venta_2011["Date"]).year
venta_2011=venta_2011[venta_2011["año"]==2011]
venta_2011=venta_2011.iloc[:,[0,3,5]]
venta_2011= venta_2011.groupby(["Store","Type"]).sum().reset_index()
valor=venta_2011.groupby("Type")["Weekly_Sales"].agg(["mean","std","count"])
print(valor)
proma=7.173672e+07
promb=4.259524e+07
promc=2.431218e+07
desva=2.659460e+07
desvb=2.017713e+07 
desvc=6.197005e+06 
counta=22
countb=17
countc=6

z1=(proma-promb)/((desva**2/counta)+(desvb**2/countb))**.5
z2=(proma-promc)/((desva**2/counta)+(desvc**2/countc))**.5
z3=(promc-promb)/((desvc**2/countc)+(desvb**2/countb))**.5

valorp1= vale.norm.sf(abs(z1))*2
valorp2= vale.norm.sf(abs(z2))*2
valorp3= vale.norm.sf(abs(z3))*2
print(valorp1)
print(valorp2)
print(valorp3)
                                                     


Determina, para cada departamento, el número de tiendas en donde este fue considerado el mejor en ventas de entre todos los departamentos en todo el 2011. ¿Existe mucha variabilidad en este dato?

In [ ]:
departamento_2011= sales_data.merge(stores_data, on="Store")
departamento_2011["año"]=pd.DatetimeIndex(departamento_2011["Date"]).year
departamento_2011=departamento_2011[departamento_2011["año"]==2011]
departamento_2011=departamento_2011.iloc[:,[0,1,3]]
departamento_2011=departamento_2011.groupby(["Store","Dept"]).sum().reset_index()
mejores_ventas=pd.DataFrame()
tiendas= departamento_2011["Store"].unique()

for stores in tiendas :
        dfcache= departamento_2011[departamento_2011["Store"]== stores]
        dfcache= dfcache.sort_values(by="Weekly_Sales", ascending=False).head(1)
        mejores_ventas=mejores_ventas.append(dfcache)
        #print(dfcache)
agrupar=mejores_ventas.groupby(["Dept"])["Store"].count()
print(agrupar)

## ¿Existe una relación entre las ventas totales de las tiendas y el tamaño de la tienda? Genera un diagrama de dispersión acerca de este, y calcula el coeficiente de correlación.

In [ ]:
venta_tamaño=sales_data[(sales_data["Date"]<="2012-12-31")&(sales_data["Date"]>="2012-01-01")]
ventas_2012=venta_tamaño.groupby(["Store"])["Weekly_Sales"].sum()
tamaño_tiendas= stores_data.merge(ventas_2012,on="Store")



print(tamaño_tiendas)

In [ ]:
sns.scatterplot(x="Size",y="Weekly_Sales",hue="Type",data=tamaño_tiendas)
vale.pearsonr(tamaño_tiendas["Size"],tamaño_tiendas["Weekly_Sales"])

 Considera únicamente los departamentos con venta estable durante todas las semanas de estudio. Con respecto a cada evento importante, calcula el incremento en % de ventas durante esas semanas con respecto al promedio global (es decir, para este departamento, qué tanto % incrementó de ventas en Super Bowl, qué tanto % en Labor Day, qué tanto % en Thanksgiving, y qué tanto % en Christmas). ¿Cuáles son los departamentos con mayor impacto por este tipo de estacionalidad?

In [ ]:
departamentos_estables=sales_data.iloc[:,[1,2,3]]
departamentos_estables=departamentos_estables.groupby(["Dept","Date"]).sum().reset_index()
departamentos_estables=departamentos_estables.pivot(index="Date",columns="Dept",values="Weekly_Sales")


logicos_estables=departamentos_estables.isna().sum()==0
departamentos_final=departamentos_estables.loc[:,logicos_estables].columns


print(departamentos_final)

In [ ]:
holidays_data=features_holidays.merge(sales_data.iloc[:,[0,1,2,3]], on= ["Store","Date"])
holidays_data=holidays_data[holidays_data["Dept"].isin(departamentos_final)]
holidays_data=holidays_data.groupby(["Dept","Date","superbowl","laborday","thanksgiving","christmas"])["Weekly_Sales"].sum().reset_index()

contar_1=holidays_data[holidays_data["superbowl"]]
print(contar_1)


In [ ]:
promedios_dept= holidays_data.groupby(["Dept"])["Weekly_Sales"].mean().reset_index(name="Promedio")
venta_super=holidays_data[holidays_data["superbowl"]]
#venta_super=venta_super[venta_super["Dept"]==1]
venta_labor=holidays_data[holidays_data["laborday"]]
venta_thanks=holidays_data[holidays_data["thanksgiving"]]
venta_navidad=holidays_data[holidays_data["christmas"]]

venta_super=venta_super.groupby(["Dept"])["Weekly_Sales"].mean().reset_index(name="superbowl")
venta_labor=venta_labor.groupby(["Dept"])["Weekly_Sales"].mean().reset_index(name="laborday")
venta_thanks=venta_thanks.groupby(["Dept"])["Weekly_Sales"].mean().reset_index(name="thanksgiving")
venta_navidad=venta_navidad.groupby(["Dept"])["Weekly_Sales"].mean().reset_index(name="navidad")

df_prom=promedios_dept.merge(venta_super,on="Dept")
df_prom=df_prom.merge(venta_labor,on="Dept")
df_prom=df_prom.merge(venta_thanks,on="Dept")
df_prom=df_prom.merge(venta_navidad,on="Dept")

df_prom["inc_superbowl"]=(df_prom["superbowl"]/df_prom["Promedio"])-1
df_prom["inc_laborday"]=(df_prom["laborday"]/df_prom["Promedio"])-1
df_prom["inc_thanksgiving"]=(df_prom["thanksgiving"]/df_prom["Promedio"])-1
df_prom["inc_navidad"]=(df_prom["navidad"]/df_prom["Promedio"])-1
df_prom=df_prom.sort_values(by="inc_thanksgiving",ascending=False)
print(df_prom)

Calcula el coeficiente de correlación de ventas por departamento con respecto a la temperatura, englobando los datos de todas las tiendas. ¿Existen algunos departamentos donde existe más relación de temperatura que otras? ¿Esto se podría ver reflejado en menor ventas para tiendas más frías/más calurosas?


In [ ]:
#print(features_data)
#print(sales_data)

all_sales=sales_data.merge(features_data,on=["Store","Date"],how= "inner")
ventas_temp=all_sales.iloc[:,[1,3,5]]
#ventas_temp=ventas_temp[ventas_temp["Dept"]==1]
#ventas_temp.plot.scatter(x="Temperature",y="Weekly_Sales")
#plt.show
#vale.pearsonr(ventas_temp["Temperature"],ventas_temp["Weekly_Sales"])
print(ventas_temp)

In [ ]:
Coef_corr=[]
departamentos_final


#print(ventas_temp)

for iteracion in departamentos_final.tolist():
    #print(iteracion)
    cor_cache=ventas_temp[ventas_temp["Dept"]==iteracion]
    #print(cor_cache)
    pearson_value=vale.pearsonr(cor_cache["Temperature"],cor_cache["Weekly_Sales"])[0]
    #print(pearson_value)
    Coef_corr.append(pearson_value)
print(Coef_corr)
    
        

In [ ]:
diccionario={"Dept":departamentos_final,"Correlacion":Coef_corr}
df_correlacion= pd.DataFrame(data=diccionario)
df_correlacion=df_correlacion.sort_values("Correlacion", ascending = False)
print(df_correlacion)

Corrobora la correlación de las ventas totales por tienda con las variables: CPI, Price_Fuel, y Markdowns. ¿Estas variables tienen poder predictivo para predicir el comportamiento de las ventas?

In [ ]:
total_tienda= all_sales.iloc[:,[0,2,3,6,12]]
total_ventas=total_tienda.groupby(["Store","Date"]).sum()["Weekly_Sales"].reset_index()

total_ventas=total_ventas.merge(total_tienda.iloc[:,[0,1,4,3]],on=["Store","Date"]).drop_duplicates()
print(vale.pearsonr(total_ventas["Weekly_Sales"],total_ventas["CPI"])[0])
print(vale.pearsonr(total_ventas["Weekly_Sales"],total_ventas["Fuel_Price"])[0])
print(total_ventas)

total_ventas.corr()



## fase 3

Dado el análisis hasta ahorita, determina cuáles son las variables que pueden ayudar a predecir la venta.

Prepara la información de manera que, en una sola tabla, se tenga la venta histórica por tienda junto con las variables que se determinaron como importantes.

In [ ]:
all_info=sales_data.merge(features_data, on=["Store","Date"])
all_info=all_info.merge(stores_data,on="Store")
all_info=all_info.iloc[:,[0,1,2,3,5,15,16]]
all_info=all_info.merge(fechas_importancia,on="Date")

#concatenar=pd.concat([tipo,depa],axis=1)
#print(datos_x)
#print(depa)

Divide el conjunto de datos en dos: uno de entrenamiento (con los años 2010 y 2011), y uno de prueba (con el año 2012).


In [ ]:
all_1011=all_info[(all_info["Date"]>="2010-01-01") & (all_info["Date"]<"2012-01-01")]
tipo=pd.get_dummies(all_1011["Type"])
depa=pd.get_dummies(all_1011["Dept"])
sp=all_1011["superbowl"]+0
ld=all_1011["laborday"]+0
tg=all_1011["thanksgiving"]+0
ch=all_1011["christmas"]+0
#print(all_info)

datos_x=pd.concat([all_1011.iloc[:,[4,6]],tipo,depa,sp,ld,tg,ch],axis=1)
#all_2012=all_info[(all_info["Date"]>="2012-01-01")] 
print(datos_x)

Entrena el modelo de regresión lineal con los datos de entrenamiento. Interpreta los coeficientes, y obtén las métricas de error promedio y el coeficiente de determinación.

In [ ]:
regresor=linear_model.LinearRegression()
x=datos_x.values.reshape(-1,90)
y=all_1011["Weekly_Sales"].values.reshape(-1,1)


regresor.fit(x,y)
prediccion=regresor.predict(x).flatten()
#print(regresor.coef_)
print(prediccion)

In [ ]:
all_1011["prediccion"]= prediccion
print(all_1011.iloc[:,[0,1,2,3,11]])

all_1011.plot.scatter("Weekly_Sales","prediccion")

r2=vale.pearsonr(all_1011["Weekly_Sales"],all_1011["prediccion"])[0]

r2=r2*r2
print(r2)

In [ ]:
error= all_1011["Weekly_Sales"]- all_1011["prediccion"]
prom_error= error.abs().mean()
errorp=error/all_1011["Weekly_Sales"]
errorp_mean= np.nanmean(errorp.abs().replace(np.inf,np.nan))
print(errorp_mean)

## Fase 4 


prediccion a nivel Mensual

In [ ]:
#print(all_info)
all_info["año"]=pd.DatetimeIndex(all_info["Date"]).year
all_info["mes"]=pd.DatetimeIndex(all_info["Date"]).month
all_info_ventas=all_info.groupby(["Store","Dept","año","mes"]).sum().reset_index()
all_info_temp=all_info.groupby(["Store","Dept","año","mes"]).mean().reset_index()

all_info_ventas=all_info_ventas.iloc[:,[0,1,2,3,4,7,8,9,10]]
all_info_temp=all_info_temp.iloc[:,[5,6]]

lista_req=pd.concat([all_info_ventas,all_info_temp],axis=1)
tienda1_depa5=lista_req[lista_req["Store"]==1]
tienda1_depa5=tienda1_depa5[tienda1_depa5["Dept"]==5]
tienda1_depa5["tiempo"]= tienda1_depa5["año"]+(tienda1_depa5["mes"]-1)/12
tienda1_depa5=tienda1_depa5.sort_values(["Weekly_Sales"],ascending=True)
#año_bajo= lista_req.sort_values(["Weekly_Sales"],ascending=True)
#tienda1_depa5.plot.line(x="tiempo",y="Weekly_Sales")

print(tienda1_depa5)

In [ ]:
lista_req_limpio=lista_req[~(((lista_req["año"]==2010) & (lista_req["mes"]== 1)) | ((lista_req["año"]==2012) & (lista_req["mes"]>10)))]
lista_req_limpio["Dept"]=lista_req_limpio["Dept"].astype(str)


lista_req_limpio=pd.get_dummies(lista_req_limpio)

lista_req_limpio["superbowl"]=lista_req_limpio["superbowl"]+0
lista_req_limpio["laborday"]=lista_req_limpio["laborday"]+0
lista_req_limpio["thanksgiving"]=lista_req_limpio["thanksgiving"]+0
lista_req_limpio["christmas"]=lista_req_limpio["christmas"]+0
lista_req_limpio=lista_req_limpio[lista_req_limpio["Dept_99"]==0]
#print(lista_req_limpio)

datos_x=lista_req_limpio.drop(["Weekly_Sales"],axis=1)
datos_y=lista_req_limpio.iloc[:,[3]]

print(datos_y)

In [ ]:
regresor_1=linear_model.LinearRegression()
regresor_1.fit(datos_x,datos_y)
prediccion_1=regresor_1.predict(datos_x).flatten()
#print(regresor.coef_)
#print(prediccion)


lista_pred=lista_req_limpio.copy()
lista_pred["prediccion"]=prediccion_1

#especifico=lista_pred[lista_pred["Store"]==2]
#especifico=especifico[especifico["Dept_2"]==1]
#especifico["tiempo"]= especifico["año"]+(especifico["mes"]-1)/12
#especifico.plot.line(x="tiempo",y="Weekly_Sales")
#especifico.plot.line(x="tiempo",y="prediccion")
#plt.plot(especifico["tiempo"].values,especifico["Weekly_Sales"].values)
#plt.plot(especifico["tiempo"].values,especifico["prediccion"].values)
#print(especifico)

In [ ]:
rcuada=vale.pearsonr(all_1011["Weekly_Sales"],all_1011["prediccion"])[0]

## Fase 5


In [ ]:
print(lista_pred)

In [ ]:
filtrado_sele=lista_pred[(lista_pred["Store"]==2) & (lista_pred["Dept_80"]==1)]
filtrado_sele["tiempo"]= filtrado_sele["año"]+(filtrado_sele["mes"]-1)/12
filtrado_sele=filtrado_sele.drop(["prediccion"],axis=1)
plt.plot(filtrado_sele["tiempo"].values,filtrado_sele["Weekly_Sales"].values)
print(filtrado_sele)



In [ ]:
filtrado_t2d2=filtrado_sele.iloc[:,[3,4,5,6,7,8,91]]
datos_x=filtrado_t2d2.drop(["Weekly_Sales"],axis=1)
datos_y=filtrado_t2d2.iloc[:,[0]]


regresor_2=linear_model.LinearRegression()
regresor_2.fit(datos_x,datos_y)
prediccion_2=regresor_2.predict(datos_x).flatten()
filtrado_t2d2["prediccion"]=prediccion_2
print(filtrado_t2d2)
plt.plot(filtrado_t2d2["tiempo"].values,filtrado_t2d2["Weekly_Sales"].values)
plt.plot(filtrado_t2d2["tiempo"].values,filtrado_t2d2["prediccion"].values)


In [ ]:
lista_combinaciones=all_info[["Store","Dept"]].drop_duplicates()
#print(lista_combinaciones)
print(lista_pred)
df_total=pd.DataFrame()

for iteracion, combinacion in lista_combinaciones.iterrows():
    tienda= combinacion["Store"]
    depa=combinacion["Dept"]
    df_filtrado=lista_pred
    df_filtrado=df_filtrado[(df_filtrado["Store"]==tienda)&(df_filtrado["Dept_"+str(depa)]==1)]
    df_filtrado=df_filtrado.iloc[:,[3,4,5,6,7,8]]
    if df_filtrado.shape[0]>10:
    
        datos_x=df_filtrado.drop(["Weekly_Sales"],axis=1)
        datos_y=df_filtrado.iloc[:,[0]]
        regresor_f=linear_model.LinearRegression()
        regresor_f.fit(datos_x,datos_y)
        prediccion_f=regresor_f.predict(datos_x).flatten()
        df_filtrado["prediccion"]=prediccion_f
        df_filtrado["Dept"]=depa
        df_total=df_total.append(df_filtrado)


In [ ]:
#print(df_total.prediccion)
lista_pred_mes= lista_pred.copy()
lista_pred_mes["mes2"]=lista_pred_mes["mes"].astype(str)
lista_pred_mes= pd.get_dummies(lista_pred_mes)
#print(lista_pred_mes)

lista_2010=lista_pred_mes[(lista_pred_mes["año"]<2012)]
lista_2010["tiempo"]=lista_2010["año"]+(lista_2010["mes"]-1)/12
lista_2012=lista_pred_mes[lista_pred_mes["año"]==2012]
lista_2012["tiempo"]=lista_2012["año"]+(lista_2012["mes"]-1)/12
df_total2=pd.DataFrame()
df_prediccion=pd.DataFrame()



for iteracion, combinacion in lista_combinaciones.iterrows():
    tienda= combinacion["Store"]
    depa=combinacion["Dept"]
    df_filtrado=lista_2010
    df_filtrado2=lista_2012
    df_filtrado2=df_filtrado2[(df_filtrado2["Store"]==tienda)&(df_filtrado2["Dept_"+str(depa)]==1)]
    df_filtrado2=df_filtrado2.iloc[:,[0,3,8,92,93,94,95,96,97,98,99,100,101,102,103,104]]
    df_filtrado=df_filtrado[(df_filtrado["Store"]==tienda)&(df_filtrado["Dept_"+str(depa)]==1)]
    df_filtrado=df_filtrado.iloc[:,[0,3,8,92,93,94,95,96,97,98,99,100,101,102,103,104]]

    if df_filtrado.shape[0]>10 and df_filtrado2.shape[0]==10:
    
        datos_x=df_filtrado.drop(["Weekly_Sales"],axis=1)
        datos_y=df_filtrado.iloc[:,[1]]
        datos_x2=df_filtrado2.drop(["Weekly_Sales"],axis=1)
        regresor_f=linear_model.LinearRegression()
        regresor_f.fit(datos_x,datos_y)
        prediccion_f=regresor_f.predict(datos_x).flatten()
        df_filtrado["prediccion"]=prediccion_f
        df_filtrado["Dept"]=depa
        df_total2=df_total2.append(df_filtrado)
        prediccion_2012=regresor_f.predict(datos_x2).flatten()
        df_filtrado2["prediccion"]=prediccion_2012
        df_filtrado2["Dept"]=depa
        df_prediccion=df_prediccion.append(df_filtrado2)

        
        #print(lista_2012)
print(df_filtrado.columns)
        

In [ ]:
df_total2["error"]=np.abs(df_total2["Weekly_Sales"]-df_total2["prediccion"])
df_prediccion["error"]=np.abs(df_prediccion["Weekly_Sales"]-df_prediccion["prediccion"])
df_total2["error%"]=np.abs(df_total2["error"]/df_total2["Weekly_Sales"])
df_prediccion["error%"]=np.abs(df_prediccion["error"]/df_prediccion["Weekly_Sales"])

error10= df_total2[df_total2["Weekly_Sales"]>0]
error12= df_prediccion[df_prediccion["Weekly_Sales"]>0]
error2010= error10["error%"].mean()
error2012= error12["error%"].mean()

#print(error2010)
#print(lista_2010)


media_td=df_total2.groupby(["Store","Dept"])[["Weekly_Sales","prediccion","error%"]].mean().sort_values("error%",ascending=False)
media_td2=df_prediccion.groupby(["Store","Dept"])[["Weekly_Sales","prediccion","error%"]].mean().sort_values("error%",ascending=True)
#print(error2010)
#print(error2012)

#print(datos_y)
print(media_td)
print(media_td2)

In [ ]:
graficat11d19=df_total2[(df_total2["Store"]==13)&(df_total2["Dept"]==45)]
grafica2012=df_prediccion[(df_prediccion["Store"]==13)&(df_prediccion["Dept"]==45)]
#print(grafica2012)
#print(graficat11d19)
#sns.lineplot(x="tiempo",y="prediccion", data= graficat11d19)
sns.lineplot(x="tiempo",y="prediccion", data= grafica2012)
#sns.lineplot(x="tiempo",y="Weekly_Sales", data= graficat11d19)
sns.lineplot(x="tiempo",y="Weekly_Sales", data= grafica2012)


In [ ]:
print(lista_pred_mes.columns)

In [ ]:
#print(df_prediccion)(
#lol= df_prediccion.sort_values(by="prediccion",ascending=False).head(5)
#print(lol)

df_prediccion.plot.scatter("prediccion","Weekly_Sales")

In [ ]:
print(datos_x.columns)
print(datos_x2.columns)